## 自动微分

### 0. Remark

- PyTorch中, 要想使`x`支持求导，必须让`x`为浮点类型.
- 在PyTorch里面，默认是标量对标量，或者标量对向量/矩阵求导！


### 1.梯度

我们可以连结一个多元函数对其所有变量的偏导数，以得到该函数的*梯度*（gradient）向量。
具体而言，设函数$f:\mathbb{R}^n\rightarrow\mathbb{R}$的输入是
一个$n$维向量$\mathbf{x}=[x_1,x_2,\ldots,x_n]^\top$，并且输出是一个标量。
函数$f(\mathbf{x})$相对于$\mathbf{x}$的梯度是一个包含$n$个偏导数的向量:

$$\nabla_{\mathbf{x}} f(\mathbf{x}) = \bigg[\frac{\partial f(\mathbf{x})}{\partial x_1}, \frac{\partial f(\mathbf{x})}{\partial x_2}, \ldots, \frac{\partial f(\mathbf{x})}{\partial x_n}\bigg]^\top,$$

其中$\nabla_{\mathbf{x}} f(\mathbf{x})$通常在没有歧义时被$\nabla f(\mathbf{x})$取代。

假设$\mathbf{x}$为$n$维向量，在`分母布局`下有:

* 对于所有$\mathbf{A} \in \mathbb{R}^{m \times n}$，都有$\nabla_{\mathbf{x}} \mathbf{A} \mathbf{x} = \mathbf{A}^\top$
* 对于所有$\mathbf{A} \in \mathbb{R}^{n \times m}$，都有$\nabla_{\mathbf{x}} \mathbf{x}^\top \mathbf{A}  = \mathbf{A}$
* 对于所有$\mathbf{A} \in \mathbb{R}^{n \times n}$，都有$\nabla_{\mathbf{x}} \mathbf{x}^\top \mathbf{A} \mathbf{x}  = (\mathbf{A} + \mathbf{A}^\top)\mathbf{x}$
* $\nabla_{\mathbf{x}} \|\mathbf{x} \|^2 = \nabla_{\mathbf{x}} \mathbf{x}^\top \mathbf{x} = 2\mathbf{x}$

同样，对于任何矩阵$\mathbf{X}$，都有$\nabla_{\mathbf{X}} \|\mathbf{X} \|_F^2 = 2\mathbf{X}$。
正如我们之后将看到的，梯度对于设计深度学习中的优化算法有很大用处。


### 2. AutoGrad in Pytorch

作为一个演示例子，(**假设我们想对函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于列向量$\mathbf{x}$求导**)。
首先，我们创建变量`x`并为其分配一个初始值。

In [1]:
import torch

In [ ]:
x = torch.arange(4.0)
print(f'x is: {x}')
print('--------------------------------')
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
print(f'x.grad is: {x.grad}')  # 默认值是None
print('--------------------------------')
y = 2 * torch.dot(x, x)
print(f'y is: {y}')
print('--------------------------------')
y.backward()
print(f'y关于x每个分量的梯度: {x.grad} ')

x is: tensor([0., 1., 2., 3.], requires_grad=True)
--------------------------------
x.grad is: None
--------------------------------
y is: 28.0
--------------------------------
y关于x每个分量的梯度: tensor([ 0.,  4.,  8., 12.]) 


现在计算`x`的另一个函数。在默认情况下，PyTorch会累积梯度，我们需要清除之前的值

In [3]:
x.grad.zero_()   # 清除之前的值
y = x.sum()
y.backward()
print(f'y关于x每个分量的梯度: {x.grad} ')

y关于x每个分量的梯度: tensor([1., 1., 1., 1.]) 


### 3.非标量变量的反向传播

`backward()`默认输出为标量,这是因为我们在深度学习中最后的loss为一个标量,这是常用的方式.   <p>

当输出为向量或矩阵时,需要通过地一个参数`gradient`来实现. `gradient`参数的维度与最终的输出(y)要保持一致. <p>
------------------------------------------------------------------------------------------------------------------   <p>

See [pytorch中backward()函数与gradient 参数详解](https://blog.csdn.net/qq_35629563/article/details/130684065) for details

In [4]:
#使用使用y.backward对非标量进行反向传播
x2 = torch.tensor([[0.0, 0.0], [1.0, 2.0]], requires_grad=True) #设定x2为求导对象

A = torch.tensor([[1.0,2.0],[3.0,4.0]])

y2 = A@x2

gradient = torch.tensor([[1.0,1.0],[2.0,1.0]]) 

y2.backward(gradient=gradient) #反向传播

res_t_gradient = x2.grad #对x2求导
res = res_t_gradient@(torch.inverse(gradient))

print("x2:\n", x2)
print('--------------------------------')
print("y2:\n", y2)
print('--------------------------------')
print("result times gradient:\n", res_t_gradient)
print('--------------------------------')
print("res:\n", res)

x2:
 tensor([[0., 0.],
        [1., 2.]], requires_grad=True)
--------------------------------
y2:
 tensor([[2., 4.],
        [4., 8.]], grad_fn=<MmBackward0>)
--------------------------------
result times gradient:
 tensor([[ 7.,  4.],
        [10.,  6.]])
--------------------------------
res:
 tensor([[1., 3.],
        [2., 4.]])


在数学上，如果我们有向量值函数 $\vec{y} = f(\vec{x})$ ，且 $\vec{y}$ 关于 $\vec{x}$ 的梯度是一个雅可比矩阵(Jacobian matrix)：

$J = \begin{pmatrix} \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}} \\ \vdots & \ddots & \vdots \\ \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}} \end{pmatrix}$

一般来说，`torch.autograd`就是用来计算vector-Jacobian product的工具。也就是说，给定任一向量 $v=(v_{1}\;v_{2}\;\cdots\;v_{m})^{T}$ ，计算 $v^{T}\cdot J$ ，如果 $v$ 恰好是标量函数 $l=g(\vec{y})$ 的梯度，也就是说 $v=(\frac{\partial l}{\partial  y_{1}}\;\cdots\;\frac{\partial l}{\partial  y_{m}})^{T}$，那么根据链式法则，vector-Jacobian product 是 $l$ 关于 $\vec{x}$ 的梯度：

$J^{T}\cdot v = \begin{pmatrix} \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}} \\ \vdots & \ddots & \vdots \\ \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}} \end{pmatrix} \begin{pmatrix} \frac{\partial l}{\partial y_{1}}\\ \vdots \\ \frac{\partial l}{\partial y_{m}} \end{pmatrix} = \begin{pmatrix} \frac{\partial l}{\partial x_{1}}\\ \vdots \\ \frac{\partial l}{\partial x_{n}} \end{pmatrix}$

（注意，$v^{T}\cdot J$ 给出了一个行向量，可以通过 $J^{T}\cdot v$ 将其视为列向量）

vector-Jacobian product 这种特性使得将外部梯度返回到具有非标量输出的模型变得非常方便。

### 4. 分离计算

有时，我们希望将某些计算移动到记录的计算图之外。 例如，假设y是作为x的函数计算的，而z则是作为y和x的函数计算的。 想象一下，我们想计算z关于x的梯度，但由于某种原因，希望将y视为一个常数， 并且只考虑到x在y被计算后发挥的作用。

In [5]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
print(x.grad == u)

x.grad.zero_()
y.sum().backward()
print(x.grad == 2 * x)

tensor([True, True, True, True])
tensor([True, True, True, True])


### 5. with torch.no_grad()

在深度学习的世界里，模型训练与评估是两个相互独立却又紧密相连的过程。训练时我们需要梯度来更新模型参数，但在评估阶段，梯度计算则成为了不必要的负担。`torch.no_grad()`正是为此而生——它允许我们在不记录梯度的情况下执行前向传播，从而节省内存并加速推理过程。

In [8]:
# 创建一个需要梯度计算的张量
x = torch.tensor([3.0], requires_grad=True)
y = torch.tensor([2.0], requires_grad=True)

# 默认情况下，计算会记录梯度信息
z = x * y
z.backward()
print(x.grad) # 输出: tensor([2.])

# 使用 torch.no_grad() 避免梯度记录
with torch.no_grad():
    z = x * y
print(z.requires_grad) # 输出: False

tensor([2.])
False
